# Comparison of the snia_toy_06 model #

In [90]:
import os, itertools
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_5, colorblind
from glob import glob

from bokeh.layouts import layout, row
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import curdoc
from bokeh.plotting import figure

In [87]:
def read_magnitudes(fname):
    COL_NAMES = ['time', 'U', 'B', 'V', 'R', 'I', 'J', 'H', 'K']
    mag_table = pd.read_csv(fname, delim_whitespace=True, comment='#', names=COL_NAMES)
    mag_table.iloc[:, 1:] = mag_table.iloc[:,1:].replace(99.999, np.nan)
    return mag_table
    
def read_spectrum(fname):
    spectrum = pd.read_csv(fname, delim_whitespace=True, comment='#', header=None)
    return spectrum

In [88]:
# uncomment the following line to download the code comparison stuff
#!wget http://opensupernova.org/~wkerzend/files/ccompare1_models.tgz
#!tar zxf ccompare1_models.tgz

**Run interactively on Google Colab on the following link**

https://colab.research.google.com/github/sn-rad-trans/code-comparison1/blob/master/plotting_snia_toy06.ipynb

In [85]:
CODE_NAMES = np.sort([os.path.basename(fname).split('_')[-1][:-4] for fname in glob('toy06/lbol_edep_toy06_*.txt')])
CODE_COLORS = {name:color for name, color in zip(CODE_NAMES, colorblind['Colorblind'][8])}

## Comparison of pseudo bolometric luminosity ##

Integration of the synthetic spectra excluding gamma rays ($>50 Angstrom$)

In [91]:

output_notebook()

# create a color iterator
colors = itertools.cycle(palette)


# create a new plot with a title and axis labels
fig_lbol  = figure(title="Comparison of Lbol", x_axis_label='time [days]', y_axis_label='Log Lbol')
fig_edep  = figure(title="Comparison of energy deposition", x_axis_label='time [days]', y_axis_label='Log Edeptot')

log_lbol_data = {}
for fname, color in zip(glob('toy06/lbol_edep_toy06_*.txt'), colors):
    name = os.path.basename(fname).split('_')[-1][:-4]
    lbol = pd.read_csv(fname, delim_whitespace=True, comment='#')
    lbol.columns = ['time', 'lbol', 'edep']
    lbol['log_lbol'] = np.log10(lbol.lbol)
    lbol['log_edep'] = np.log10(lbol.edep)
    #lbol.plot_bokeh()
    # add a line renderer with legend and line thickness
    fig_lbol.line(lbol.time, lbol.log_lbol, legend=name, line_width=2, color=CODE_COLORS[name])
    fig_edep.line(lbol.time, lbol.log_edep, legend=name, line_width=2, color=CODE_COLORS[name])
    log_lbol_data[name] = lbol

fig_lbol.legend.location = "top_right"
fig_lbol.legend.click_policy="hide"
fig_edep.legend.location = "top_right"
fig_edep.legend.click_policy="hide"

# show the results
show(row([fig_lbol, fig_edep]))#

Loading BokehJS ...

## Comparing the magnitudes ##

In [92]:
## Reading the data
mag_data = {}
for fname, color in zip(glob('toy06/mags_toy06_*'), colors):
    name = os.path.basename(fname).split('_')[-1][:-4]
    mag_data[name] = read_magnitudes(fname)
#    lbol.columns = ['time', 'lbol', 'edep']
#    lbol['log_lbol'] = np.log10(lbol.lbol)
    #lbol.plot_bokeh()
    # add a line renderer with legend and line thickness
#    p.line(lbol.time, lbol.log_lbol, legend=name, line_width=2, color=color)

In [93]:
output_notebook()

MAG_NAMES = list(mag_data.values())[0].columns[1:]
all_tabs = []
for mag in MAG_NAMES:
    fig = figure()
    for name, mag_table in mag_data.items():
        fig.line(mag_table.time, mag_table[mag], legend=name, line_width=2, color=CODE_COLORS[name])
        fig.y_range.flipped = True
    fig.legend.location = "top_right"
    fig.legend.click_policy="hide"
    lout = layout([[fig]])
    tab = Panel(child=lout,title=mag)
    all_tabs.append(tab)
        
show(Tabs(tabs=all_tabs))

Loading BokehJS ...